In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from keras.models import load_model, Model

Using TensorFlow backend.


In [2]:
sales_eval = pd.read_csv('sales_train_evaluation.csv')
sales_eval.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [3]:
sales_eval.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1, inplace=True)
sales_eval.head()

,id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [4]:
sales_valid = pd.read_csv('sales_train_validation.csv')
sales_valid.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [5]:
sales_valid.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1, inplace=True)
sales_valid.set_index('id', inplace=True)
sales_valid.head()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [6]:
sub = pd.read_csv('sample_submission.csv')
sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
sub.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60979,FOODS_3_827_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Multiple Output Strategy
# X we will keep all columns execpt the last 28 columns 
X_train = np.expand_dims(sales_valid.values[:,:-28],axis = 2)
# the last 28 columns are our prediction
y_train = sales_valid.values[:,-28:]

# for test we keep all the columns from validation data to predict the next 28 days sales
X_test = np.expand_dims(sales_valid.values[:,28:1913],axis = 2)

# lets have a look on the shape 
print(X_train.shape,y_train.shape,X_test.shape)

(30490, 1885, 1) (30490, 28) (30490, 1885, 1)


In [9]:
nsamples, nx, ny = X_train.shape
X2_train = X_train.reshape((nsamples,nx*ny))
nsamples, nx, ny = X_test.shape
X2_test = X_test.reshape((nsamples,nx*ny))

In [10]:
X2_train.shape

(30490, 1885)

In [11]:
sc = MinMaxScaler(feature_range = (0, 1))
X_train_scaled = sc.fit_transform(X2_train)
X_test_scaled = sc.fit_transform(X2_test)

In [16]:
X_train_0 = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_0 = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

In [17]:
X_train_0.shape

(30490, 1885, 1)

In [12]:
sales_model = Sequential()
sales_model.add(LSTM(units = 100, input_shape = (1885,1)))
sales_model.add(Dropout(0.5))
#sales_model.add(LSTM(units = 400))
#sales_model.add(Dropout(0.2))

sales_model.add(Dense(28))

sales_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['accuracy'])
sales_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 28)                2828      
Total params: 43,628
Trainable params: 43,628
Non-trainable params: 0
_________________________________________________________________


In [18]:
sales_model.fit(X_train_0,y_train,batch_size = 64,epochs = 8)

Epoch 1/8
30490/30490 [==============================] - 2263s 74ms/step - loss: 13.1906 - accuracy: 0.0472
Epoch 2/8
30490/30490 [==============================] - 2290s 75ms/step - loss: 12.0417 - accuracy: 0.0508
Epoch 3/8
30490/30490 [==============================] - 2288s 75ms/step - loss: 13.3556 - accuracy: 0.0493
Epoch 4/8
30490/30490 [==============================] - 2177s 71ms/step - loss: 12.9293 - accuracy: 0.0525
Epoch 5/8
30490/30490 [==============================] - 2208s 72ms/step - loss: 12.7192 - accuracy: 0.0553
Epoch 6/8
30490/30490 [==============================] - 2291s 75ms/step - loss: 12.6704 - accuracy: 0.0561
Epoch 7/8
30490/30490 [==============================] - 2258s 74ms/step - loss: 12.0871 - accuracy: 0.0582
Epoch 8/8
30490/30490 [==============================] - 2242s 74ms/step - loss: 12.5672 - accuracy: 0.0595


In [19]:
pred = sales_model.predict(X_test_0)

In [20]:
pred

array([[1.0725806, 1.0856825, 1.0154748, ..., 1.3010519, 1.6026236,
        1.6106066],
       [1.0592027, 1.0733051, 1.0030575, ..., 1.286988 , 1.5860068,
        1.5929906],
       [1.069864 , 1.0831548, 1.013003 , ..., 1.2981716, 1.5993291,
        1.6071265],
       ...,
       [1.0620565, 1.0759796, 1.0056415, ..., 1.2900702, 1.5894289,
        1.5967672],
       [1.0838786, 1.0960312, 1.0262574, ..., 1.3126843, 1.6171231,
        1.6258276],
       [1.0585477, 1.0727004, 1.0024264, ..., 1.2862824, 1.5851386,
        1.5921612]], dtype=float32)

In [21]:
valid = pd.DataFrame(np.array(pred).reshape(30490,28), columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6',
                                                                     'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14',
                                                                     'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
                                                                     'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'])
valid.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,1.072581,1.085683,1.015475,1.176366,1.467075,1.632779,1.776395,1.304566,1.208545,1.192326,...,1.305283,1.605240,1.629518,1.172936,1.174419,1.019648,1.132642,1.301052,1.602624,1.610607
1,1.059203,1.073305,1.003057,1.162625,1.451137,1.615005,1.759300,1.289468,1.194269,1.177744,...,1.291033,1.588585,1.611428,1.159473,1.159896,1.007093,1.119269,1.286988,1.586007,1.592991
2,1.069864,1.083155,1.013003,1.173576,1.464014,1.629193,1.773012,1.301527,1.205609,1.189294,...,1.302480,1.601881,1.625804,1.170196,1.171414,1.017022,1.130005,1.298172,1.599329,1.607126
3,1.090815,1.102553,1.032597,1.195397,1.488401,1.657393,1.799074,1.325021,1.228570,1.211904,...,1.324124,1.627721,1.654020,1.191482,1.194849,1.036734,1.151288,1.320716,1.625582,1.634908
4,1.094029,1.105353,1.035721,1.198151,1.493544,1.661245,1.804248,1.328784,1.231185,1.215165,...,1.328812,1.631825,1.658242,1.194679,1.197215,1.039317,1.154642,1.323300,1.629800,1.639223


In [22]:
valid.shape

(30490, 28)

In [23]:
sales = sales_valid.reset_index()
valid['id'] = sales['id'].values
valid.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F20,F21,F22,F23,F24,F25,F26,F27,F28,id
0,1.072581,1.085683,1.015475,1.176366,1.467075,1.632779,1.776395,1.304566,1.208545,1.192326,...,1.605240,1.629518,1.172936,1.174419,1.019648,1.132642,1.301052,1.602624,1.610607,HOBBIES_1_001_CA_1_validation
1,1.059203,1.073305,1.003057,1.162625,1.451137,1.615005,1.759300,1.289468,1.194269,1.177744,...,1.588585,1.611428,1.159473,1.159896,1.007093,1.119269,1.286988,1.586007,1.592991,HOBBIES_1_002_CA_1_validation
2,1.069864,1.083155,1.013003,1.173576,1.464014,1.629193,1.773012,1.301527,1.205609,1.189294,...,1.601881,1.625804,1.170196,1.171414,1.017022,1.130005,1.298172,1.599329,1.607126,HOBBIES_1_003_CA_1_validation
3,1.090815,1.102553,1.032597,1.195397,1.488401,1.657393,1.799074,1.325021,1.228570,1.211904,...,1.627721,1.654020,1.191482,1.194849,1.036734,1.151288,1.320716,1.625582,1.634908,HOBBIES_1_004_CA_1_validation
4,1.094029,1.105353,1.035721,1.198151,1.493544,1.661245,1.804248,1.328784,1.231185,1.215165,...,1.631825,1.658242,1.194679,1.197215,1.039317,1.154642,1.323300,1.629800,1.639223,HOBBIES_1_005_CA_1_validation


In [24]:
valid.tail()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F20,F21,F22,F23,F24,F25,F26,F27,F28,id
30485,1.066243,1.079750,1.009651,1.169763,1.459808,1.624266,1.768424,1.297408,1.201637,1.185258,...,1.597295,1.620850,1.166567,1.167338,1.013544,1.126422,1.294241,1.594844,1.602350,FOODS_3_823_WI_3_validation
30486,1.060212,1.074230,1.003997,1.163736,1.452219,1.616364,1.760406,1.290558,1.195430,1.178784,...,1.589769,1.612734,1.160499,1.161126,1.008027,1.120329,1.288134,1.587258,1.594373,FOODS_3_824_WI_3_validation
30487,1.062057,1.075980,1.005641,1.165664,1.454404,1.618807,1.762832,1.292681,1.197343,1.180924,...,1.592165,1.615282,1.162277,1.163128,1.009813,1.122038,1.290070,1.589429,1.596767,FOODS_3_825_WI_3_validation
30488,1.083879,1.096031,1.026257,1.187786,1.481400,1.647805,1.791348,1.317401,1.220384,1.204266,...,1.619338,1.644600,1.184419,1.186257,1.029872,1.144393,1.312684,1.617123,1.625828,FOODS_3_826_WI_3_validation
30489,1.058548,1.072700,1.002426,1.161972,1.450383,1.614090,1.758460,1.288722,1.193517,1.177028,...,1.587776,1.610503,1.158780,1.159180,1.006442,1.118596,1.286282,1.585139,1.592161,FOODS_3_827_WI_3_validation


In [25]:
validation = pd.merge(sales, valid, on = ['id'], how = 'left')
validation.head()

,id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,1.305283,1.605240,1.629518,1.172936,1.174419,1.019648,1.132642,1.301052,1.602624,1.610607
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,1.291033,1.588585,1.611428,1.159473,1.159896,1.007093,1.119269,1.286988,1.586007,1.592991
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,1.302480,1.601881,1.625804,1.170196,1.171414,1.017022,1.130005,1.298172,1.599329,1.607126
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,1.324124,1.627721,1.654020,1.191482,1.194849,1.036734,1.151288,1.320716,1.625582,1.634908
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,1.328812,1.631825,1.658242,1.194679,1.197215,1.039317,1.154642,1.323300,1.629800,1.639223


In [26]:
validation.set_index('id', inplace=True)
validation.head()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,...,1.305283,1.605240,1.629518,1.172936,1.174419,1.019648,1.132642,1.301052,1.602624,1.610607
HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,...,1.291033,1.588585,1.611428,1.159473,1.159896,1.007093,1.119269,1.286988,1.586007,1.592991
HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,...,1.302480,1.601881,1.625804,1.170196,1.171414,1.017022,1.130005,1.298172,1.599329,1.607126
HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,...,1.324124,1.627721,1.654020,1.191482,1.194849,1.036734,1.151288,1.320716,1.625582,1.634908
HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,...,1.328812,1.631825,1.658242,1.194679,1.197215,1.039317,1.154642,1.323300,1.629800,1.639223


In [27]:
# for eval we keep all the columns from the previous result and remove the first 56 from the sales_validation
X_test_eval = np.expand_dims(validation.values[:,56:1941],axis = 2)
print(X_test_eval.shape)

(30490, 1885, 1)


In [28]:
nsamples, nx, ny = X_test_eval.shape
X2_test_eval = X_test_eval.reshape((nsamples,nx*ny))

In [29]:
X_test_eval_scaled = sc.fit_transform(X2_test_eval)

In [30]:
X_test_1 = X_test_eval_scaled.reshape((X_test_eval_scaled.shape[0], X_test_eval_scaled.shape[1], 1))

In [31]:
pred1 = sales_model.predict(X_test_1)

In [32]:
evalu = pd.DataFrame(np.array(pred1).reshape(30490,28), columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6',
                                                                     'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14',
                                                                     'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
                                                                     'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'])
evalu.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,1.071683,1.084841,1.014699,1.175471,1.466139,1.631619,1.775282,1.303565,1.207585,1.191263,...,1.304363,1.604131,1.628242,1.172039,1.173441,1.018724,1.131850,1.300118,1.601594,1.609542
1,1.059144,1.073237,1.003013,1.162563,1.451087,1.614912,1.759203,1.289390,1.194199,1.177639,...,1.290974,1.588485,1.611321,1.159419,1.159827,1.007008,1.119239,1.286919,1.585949,1.592939
2,1.069177,1.082521,1.012362,1.172892,1.463131,1.628278,1.772067,1.300732,1.204909,1.188539,...,1.301687,1.601004,1.624858,1.169515,1.170720,1.016381,1.129330,1.297479,1.598467,1.606226
3,1.089079,1.100940,1.030938,1.193341,1.487012,1.654838,1.797598,1.323224,1.226179,1.210148,...,1.322961,1.625813,1.651746,1.189592,1.192325,1.035000,1.149361,1.318437,1.623343,1.632551
4,1.092068,1.103702,1.033731,1.196407,1.490597,1.658828,1.801426,1.326597,1.229374,1.213384,...,1.326158,1.629528,1.655783,1.192613,1.195568,1.037795,1.152374,1.321583,1.627085,1.636503


In [33]:
df_sale = sub.loc[30490:]
df_sale.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30490,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30491,HOBBIES_1_002_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30492,HOBBIES_1_003_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30493,HOBBIES_1_004_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30494,HOBBIES_1_005_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
evalu['id'] = df_sale['id'].values
evalu.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F20,F21,F22,F23,F24,F25,F26,F27,F28,id
0,1.071683,1.084841,1.014699,1.175471,1.466139,1.631619,1.775282,1.303565,1.207585,1.191263,...,1.604131,1.628242,1.172039,1.173441,1.018724,1.131850,1.300118,1.601594,1.609542,HOBBIES_1_001_CA_1_evaluation
1,1.059144,1.073237,1.003013,1.162563,1.451087,1.614912,1.759203,1.289390,1.194199,1.177639,...,1.588485,1.611321,1.159419,1.159827,1.007008,1.119239,1.286919,1.585949,1.592939,HOBBIES_1_002_CA_1_evaluation
2,1.069177,1.082521,1.012362,1.172892,1.463131,1.628278,1.772067,1.300732,1.204909,1.188539,...,1.601004,1.624858,1.169515,1.170720,1.016381,1.129330,1.297479,1.598467,1.606226,HOBBIES_1_003_CA_1_evaluation
3,1.089079,1.100940,1.030938,1.193341,1.487012,1.654838,1.797598,1.323224,1.226179,1.210148,...,1.625813,1.651746,1.189592,1.192325,1.035000,1.149361,1.318437,1.623343,1.632551,HOBBIES_1_004_CA_1_evaluation
4,1.092068,1.103702,1.033731,1.196407,1.490597,1.658828,1.801426,1.326597,1.229374,1.213384,...,1.629528,1.655783,1.192613,1.195568,1.037795,1.152374,1.321583,1.627085,1.636503,HOBBIES_1_005_CA_1_evaluation


In [35]:
cols = evalu.columns.tolist()
cols = cols[-1:] + cols[:-1]
evalu = evalu[cols]
evalu.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,1.071683,1.084841,1.014699,1.175471,1.466139,1.631619,1.775282,1.303565,1.207585,...,1.304363,1.604131,1.628242,1.172039,1.173441,1.018724,1.131850,1.300118,1.601594,1.609542
1,HOBBIES_1_002_CA_1_evaluation,1.059144,1.073237,1.003013,1.162563,1.451087,1.614912,1.759203,1.289390,1.194199,...,1.290974,1.588485,1.611321,1.159419,1.159827,1.007008,1.119239,1.286919,1.585949,1.592939
2,HOBBIES_1_003_CA_1_evaluation,1.069177,1.082521,1.012362,1.172892,1.463131,1.628278,1.772067,1.300732,1.204909,...,1.301687,1.601004,1.624858,1.169515,1.170720,1.016381,1.129330,1.297479,1.598467,1.606226
3,HOBBIES_1_004_CA_1_evaluation,1.089079,1.100940,1.030938,1.193341,1.487012,1.654838,1.797598,1.323224,1.226179,...,1.322961,1.625813,1.651746,1.189592,1.192325,1.035000,1.149361,1.318437,1.623343,1.632551
4,HOBBIES_1_005_CA_1_evaluation,1.092068,1.103702,1.033731,1.196407,1.490597,1.658828,1.801426,1.326597,1.229374,...,1.326158,1.629528,1.655783,1.192613,1.195568,1.037795,1.152374,1.321583,1.627085,1.636503


In [36]:
cols = valid.columns.tolist()
cols = cols[-1:] + cols[:-1]
valid = valid[cols]
valid.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,1.072581,1.085683,1.015475,1.176366,1.467075,1.632779,1.776395,1.304566,1.208545,...,1.305283,1.605240,1.629518,1.172936,1.174419,1.019648,1.132642,1.301052,1.602624,1.610607
1,HOBBIES_1_002_CA_1_validation,1.059203,1.073305,1.003057,1.162625,1.451137,1.615005,1.759300,1.289468,1.194269,...,1.291033,1.588585,1.611428,1.159473,1.159896,1.007093,1.119269,1.286988,1.586007,1.592991
2,HOBBIES_1_003_CA_1_validation,1.069864,1.083155,1.013003,1.173576,1.464014,1.629193,1.773012,1.301527,1.205609,...,1.302480,1.601881,1.625804,1.170196,1.171414,1.017022,1.130005,1.298172,1.599329,1.607126
3,HOBBIES_1_004_CA_1_validation,1.090815,1.102553,1.032597,1.195397,1.488401,1.657393,1.799074,1.325021,1.228570,...,1.324124,1.627721,1.654020,1.191482,1.194849,1.036734,1.151288,1.320716,1.625582,1.634908
4,HOBBIES_1_005_CA_1_validation,1.094029,1.105353,1.035721,1.198151,1.493544,1.661245,1.804248,1.328784,1.231185,...,1.328812,1.631825,1.658242,1.194679,1.197215,1.039317,1.154642,1.323300,1.629800,1.639223


In [37]:
frames = [valid, evalu]

result = pd.concat(frames)

In [38]:
result.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,1.072581,1.085683,1.015475,1.176366,1.467075,1.632779,1.776395,1.304566,1.208545,...,1.305283,1.605240,1.629518,1.172936,1.174419,1.019648,1.132642,1.301052,1.602624,1.610607
1,HOBBIES_1_002_CA_1_validation,1.059203,1.073305,1.003057,1.162625,1.451137,1.615005,1.759300,1.289468,1.194269,...,1.291033,1.588585,1.611428,1.159473,1.159896,1.007093,1.119269,1.286988,1.586007,1.592991
2,HOBBIES_1_003_CA_1_validation,1.069864,1.083155,1.013003,1.173576,1.464014,1.629193,1.773012,1.301527,1.205609,...,1.302480,1.601881,1.625804,1.170196,1.171414,1.017022,1.130005,1.298172,1.599329,1.607126
3,HOBBIES_1_004_CA_1_validation,1.090815,1.102553,1.032597,1.195397,1.488401,1.657393,1.799074,1.325021,1.228570,...,1.324124,1.627721,1.654020,1.191482,1.194849,1.036734,1.151288,1.320716,1.625582,1.634908
4,HOBBIES_1_005_CA_1_validation,1.094029,1.105353,1.035721,1.198151,1.493544,1.661245,1.804248,1.328784,1.231185,...,1.328812,1.631825,1.658242,1.194679,1.197215,1.039317,1.154642,1.323300,1.629800,1.639223


In [39]:
result.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,FOODS_3_823_WI_3_evaluation,1.065762,1.079360,1.009179,1.169378,1.459032,1.623727,1.767687,1.296871,1.201263,...,1.298040,1.596742,1.620247,1.166077,1.167012,1.013189,1.125896,1.293885,1.594205,1.601705
30486,FOODS_3_824_WI_3_evaluation,1.060119,1.074139,1.003921,1.163567,1.452258,1.616210,1.760453,1.290492,1.195239,...,1.292016,1.589702,1.612636,1.160399,1.160885,1.007918,1.120220,1.287945,1.587165,1.594230
30487,FOODS_3_825_WI_3_evaluation,1.061813,1.075707,1.005498,1.165312,1.454293,1.618466,1.762627,1.292408,1.197045,...,1.293824,1.591817,1.614920,1.162102,1.162723,1.009500,1.121922,1.289727,1.589275,1.596474
30488,FOODS_3_826_WI_3_evaluation,1.082477,1.094826,1.024772,1.186563,1.479090,1.646019,1.789121,1.315761,1.219120,...,1.315899,1.617581,1.642817,1.182926,1.185158,1.028817,1.142717,1.311482,1.615086,1.623823
30489,FOODS_3_827_WI_3_evaluation,1.058531,1.072670,1.002442,1.161931,1.450353,1.614096,1.758418,1.288697,1.193544,...,1.290321,1.587720,1.610494,1.158802,1.159161,1.006435,1.118623,1.286273,1.585184,1.592128


In [40]:
result.to_csv (r'submission4.csv', index = False, header=True)